# 

# Importing libraries

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image, ImageDraw
import random

# Data description

In [ ]:
train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
test = pd.read_csv('../input/tensorflow-great-barrier-reef/test.csv')
sample_submission = pd.read_csv('../input/tensorflow-great-barrier-reef/example_sample_submission.csv')

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
test.info()

In [ ]:
test.head()

# Data visualization

In [ ]:
df_train = train.copy()
train_dir = "../input/tensorflow-great-barrier-reef/train_images"
df_train['image_path'] = train_dir + "/video_" + df_train['video_id'].astype(str) + "/" + df_train['video_frame'].astype(str) + ".jpg"
df_train.head()

In [ ]:
df_train['video_id'].value_counts()

In [ ]:
 plt.bar(x = df_train['video_id'].value_counts().index, height = df_train['video_id'].value_counts())

In [ ]:
df_train.info()

In [ ]:
num_training_images = len(df_train)
num_training_images

In [ ]:
plt.figure(figsize = (20, 20))
for i in range(0, 10):
    plt.subplot(5, 2, i+1)
    index = random.randint(0, 23501)
    img_path = df_train['image_path'].iloc[index]
    img = Image.open(img_path)
    plt.imshow(img)

In [ ]:
img_video_2_10 = plt.imread('../input/tensorflow-great-barrier-reef/train_images/video_2/10.jpg')
img_video_2_10.shape

In [ ]:
df_train_annotated = df_train[df_train['annotations'] != '[]']
df_train_annotated

In [ ]:
(len(df_train_annotated)/ num_training_images) * 100

In [ ]:
df_train['No_bbox'] = df_train['annotations'].apply(lambda x:x.count('{'))
df_train.head()

In [ ]:
df_train['No_bbox'].value_counts()

In [ ]:
plt.figure(figsize = (10,5))
sns.countplot(x = df_train['No_bbox'])

In [ ]:
df_train_annotated['No_bbox'] = df_train_annotated['annotations'].apply(lambda x:x.count('{'))
df_train_annotated.head()

In [ ]:
df_train_annotated[df_train_annotated['No_bbox'] >= 5]

In [ ]:
df_train_annotated[df_train_annotated['No_bbox'] >= 8]

In [ ]:
import ast

ast.literal_eval(df_train_annotated.iloc[2345].annotations)

In [ ]:
def visualize_img_annots(df, id):
    img_path = df['image_path'][id]
    img = Image.open(img_path)
    bounding_boxes = ast.literal_eval(df['annotations'].loc[id])
    for box in bounding_boxes:
            shape = (box['x'], box['y'], box['x']+box['width'], box['y']+box['height'])
            ImageDraw.Draw(img).rectangle(shape, outline=180, width=3)
    display(img)

In [ ]:
visualize_img_annots(df_train_annotated, id = 9292)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
sns.set_style('darkgrid')

from PIL import Image, ImageDraw
import tensorflow as tf

import os
import ast  ## Change str -> list.
import sys
import time

import warnings
warnings.filterwarnings('ignore')

import greatbarrierreef

In [ ]:
df_train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
df_train['img_path'] = os.path.join('../input/tensorflow-great-barrier-reef/train_images')+"/video_"+df_train.video_id.astype(str)+"/"+df_train.video_frame.astype(str)+".jpg"
df_train.head()

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(df_train['video_id'], color='#2196F3')

In [ ]:
with_annotation = len(df_train[df_train['annotations'] != '[]'])
without_annotation = len(df_train[df_train['annotations'] == '[]'])

labels = ['Without Bounding Box', 'With Bounding Box']

fig = go.Figure([go.Bar(x=labels, 
                        y=[without_annotation, with_annotation], width=0.6)])

fig.update_layout(autosize=False, width=700, height=400, margin=dict(l=60, r=60, b=50, t=50, pad=4))
fig.show()

In [ ]:
# creating new column which contains the total number of bounding boxes
df_train['No_bbox'] = df_train['annotations'].apply(lambda x:x.count('{')) 

# Example

n = df_train['No_bbox'][12843]
print(df_train['annotations'][12843])
print(f'Number of bounding boxes are : {n}.')

In [ ]:
fig = px.bar(df_train['No_bbox'].value_counts().drop(0), title='Count of Bounding Boxes')
fig.update_layout(autosize=False, width=700, height=400, margin=dict(l=60, r=60, b=50, t=50, pad=4))
fig.show()


In [ ]:
df_train['annotations'] = df_train['annotations'].apply(ast.literal_eval)
df_train.head()

In [ ]:
df2 = df_train[df_train['annotations'].astype(str) != "[]"]
df2 = df2[df2['No_bbox'] == 5]
df2.head()

In [ ]:
def img_viz(df, id):
    image = df_train['img_path'][id]
    img = Image.open(image)
    
    for box in df_train['annotations'][id]:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="red", width=3)
    display(img)

In [ ]:
img_viz(df_train, id=5474)

In [ ]:
import os
import cv2
import ast
import json
import subprocess
from glob import glob
from tqdm.notebook import tqdm
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Video

In [ ]:
INPUT_PATH = '../input/tensorflow-great-barrier-reef'
HEIGHT = 720 # image height
WIDTH  = 1280 # image width

In [ ]:
df_train = pd.read_csv(INPUT_PATH + '/train.csv')
display(df_train)
print(df_train.info())

In [ ]:
for video_id in df_train['video_id'].unique():
    print(f'video_id: {video_id}')
    print(f'w   annotations:  {sum(df_train[df_train["video_id"]==video_id]["annotations"] == "[]")}')
    print(f'w/o annotations:  {sum(df_train[df_train["video_id"]==video_id]["annotations"] != "[]")}\n')

In [ ]:
# Change the type of 'annotations' from str to list
df_train['annotations'] = df_train['annotations'].apply(ast.literal_eval) # str -> list
# Add columns of image path and number of bboxes, and the difference.
df_train['image_path'] = INPUT_PATH + '/train_images/video_' + df_train['video_id'].astype(str) + '/' + df_train['video_frame'].astype(str) + ".jpg"
df_train['num_bboxes'] = df_train['annotations'].apply(lambda x: len(x))
df_train['diff_num_bboxes'] = df_train['num_bboxes'].diff().fillna(0).astype(int)
display(df_train)

In [ ]:
# Get indexes with 3 or more diff_num_bboxes
indexes = df_train[abs(df_train['diff_num_bboxes'])>=3].index.values
display(df_train.iloc[indexes])